# Callin Switzer
Read in all csv files from individual trials, and 
combine into a single long dataset


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import os
import pandas as pd
import time
import re
from datetime import datetime
import sys
import csv
import seaborn as sns
from collections import Counter
from IPython.display import Image

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

print(sys.version)

now = datetime.now()
print("last run on " + str(now))

In [ ]:
# define directories
baseDir = os.getcwd()
dataDir = '/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/SonicationBehaviorTrials_NoImages/'


In [ ]:
os.chdir(dataDir)

In [ ]:
fldrs = [f for f in os.listdir(".") if not f.startswith('.')]
len(fldrs) # should be 232

In [ ]:
fldrs.sort(key=lambda x: os.path.getmtime(x)) # sort by time created

In [ ]:
# write list of all trials
with open("/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/RawDataFilenames.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(np.transpose(np.array([fldrs])))

In [ ]:
os.chdir(dataDir)

In [ ]:
# for each folder, open it, and read the ampFreq.txt file

folders = fldrs

NoFlightFolder = "/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/SonicationBehaviorData_FlightsRemoved/"

probs = 0
for ii in range(len(folders)):

    # read contents of each folder
    fcontents = [f for f in os.listdir(folders[ii]) if not f.startswith('.')]

    # get ampFreq file(s). there should be one per folder
    ampFreqFile = [x for x in fcontents if 'ampFreq.txt' in x]

    if len(ampFreqFile) > 1: 
        print("PROBLEM " + str(ii))


    # read ampFreq file
    tmpDF = pd.read_table(os.path.join(folders[ii], ampFreqFile[0]), header = None, sep = ',')

    # add folder name to data frame
    tmpDF[7] = folders[ii]

    # add frequency information
    folderInFolder = [x for x in fcontents if not 'ampFreq.txt' in x]

    if len(folderInFolder) > 1: 
        print("PROBLEM")
        probs += 1

    # list files in inner folder
    accRecFiles = [x for x in os.listdir(os.path.join(folders[ii], folderInFolder[0])) if not x.startswith('.')]
    accRecFiles.sort()

    tmpDF[8] = accRecFiles

    # remove wingbeats
    tmpDF2 = tmpDF.loc[(tmpDF[0] > 220) & (tmpDF[0] < 450)]

    # renumber index
    tmpDF2.index = (np.arange(1, tmpDF2.shape[0] + 1))

    # write to .csv
    tmpDF2.to_csv(NoFlightFolder + folders[ii] + '.csv', 
                 header = False, index = True)
    
    if np.mod(ii, 10) == 0:
        print(ii)
    
print(str(probs) + " problems")

In [ ]:
## combine csv's all into a single file
csvDir = NoFlightFolder

csvFiles = [f for f in os.listdir(csvDir) if f.endswith('.csv')]

np_array_list = []
for file_ in csvFiles:
    df = pd.read_csv(os.path.join(csvDir, file_),index_col=None, header=None)
    np_array_list.append(df.as_matrix())

comb_np_array = np.vstack(np_array_list)
big_frame = pd.DataFrame(comb_np_array)

big_frame.columns = ['index', 'freq', 'amp', 'datetime', 'rewNum', 'rewTF', 'lowRewAmp', 'highrewAmp', 'BeeNumCol', 'accFile']

big_frame.head()

In [ ]:
big_frame["freq"].shape

In [ ]:
# plot histogram of frequencies
vls = [int(big_frame["freq"][ii]) for ii in range(len(big_frame))]
plt.hist(vls, bins = 50)
plt.show()

In [ ]:
# check to see if datetime and accFile agree
eqs = [big_frame['datetime'][ii][1:25] == big_frame['accFile'][ii][0:24] for ii in range(len(big_frame['datetime']))]

In [ ]:
np.sum(np.invert(eqs)) # should be zero if all of them match

In [ ]:
# find the one that doesn't match
big_frame.loc[np.invert(eqs)] # now they all match

### make a new column for bee color, hive, reward frequency, date, and treatment (initial, high, low)



In [ ]:
str2 = [big_frame['BeeNumCol'][ii].split("Bee")[1] for ii in range(len(big_frame['BeeNumCol']))]

In [ ]:
beeColNum = [str2[ii].split(r'_')[0] for ii in range(len(str2))]

In [ ]:
hiveNum = [str2[ii].split(r'_')[2] for ii in range(len(str2))]

In [ ]:
hive = [int(re.findall(r'\d+', ii)[0]) for ii in hiveNum]

In [ ]:
# definition for extracting trial number
def extractNum(string):
    try: 
        aa =  re.findall(r'\d+', string)[0]
    except:
        aa = "1"
    return(int(aa))
    

In [ ]:
# get trial number
trialNum = [extractNum(ii) for ii in beeColNum]

In [ ]:
# function to get only characters
def extractChar(string):
    return(" ".join(re.findall("[a-zA-Z]+", string)))

In [ ]:
# get just bee color
beeCol = [extractChar(strg) for strg in beeColNum]

In [ ]:
# get datetime
dateTime = [datetime.strptime(dt1, ' %Y_%m_%d__%H_%M_%S_%f') for dt1 in big_frame['datetime']]

In [ ]:
# string format time
dateTime_format = [datetime.strftime(datetime_object, "%Y-%m-%d %H:%M:%S.%f") for datetime_object in dateTime]

In [ ]:
# get reward frequencies
s1 = big_frame['accFile'][0]

In [ ]:
rewFrqs = [s1.split("_")[8:10] for s1 in big_frame['accFile']]


In [ ]:
rewDF = pd.DataFrame(rewFrqs, columns = ['lowFrq', 'highFrq'])

# make sure all reward frequencies are integers
rewFrqs = [[int(float(jj)) for jj in ff] for ff in rewFrqs]

In [ ]:
# add to big data frame
big_frame['hive'] = hive
big_frame['trialNum'] = trialNum
big_frame['beeCol'] = beeCol
big_frame['beeCol'] = big_frame['beeCol'].str.lower()
big_frame['datetime_str'] = dateTime_format

In [ ]:
big_frame2 = pd.concat([big_frame, rewDF], axis  = 1)

big_frame2.head()

In [ ]:
# remove test rows
big_frame2 = big_frame2.loc[big_frame2['beeCol'] != "testtestphoto",:]

In [ ]:
# fix typo -- accidentally wrote "whitred" instead of whitered
big_frame2["beeCol"] = big_frame2["beeCol"].replace("whitred", "whitered")

In [ ]:
big_frame2.shape

In [ ]:
big_frame2.to_csv('/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/CombinedBeeTrials_noWingbeats.csv', header = True, index = False)

## Incorporate IT-Span Information

In [ ]:
#%qtconsole

In [ ]:
dataDir2 = '/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/BeeMetaData/'

In [ ]:
## Add IT Span to dataset
md = pd.read_csv(dataDir2 + "BeeMetaData1.csv")
print(md.shape)
md2 = pd.read_csv(dataDir2 + "BeeMetaData2.csv")
print(md2.shape)

In [ ]:
md_comb = pd.concat([md, md2])
md_comb.shape

In [ ]:
md_comb= md_comb[pd.notnull(md_comb['IT'])]
md_comb.shape

In [ ]:
md_sub = md_comb.loc[:, ['BeeColorNum', 'IT', 'Hive']]
md_sub = md_sub.reset_index(drop=True)
md_sub['Hive'] = md_sub['Hive'].astype(int)
md_sub

In [ ]:
# check to make sure each row is unique
print(len(md_sub))
print(len(np.unique(md_sub['BeeColorNum'])))

### Merge metadata into full dataset



In [ ]:
bf2 = big_frame2.merge(md_sub, left_on =['beeCol', 'hive'], right_on = ["BeeColorNum", "Hive"], how = 'outer', indicator = True)
bf2.head()

In [ ]:
Counter(bf2._merge) # shows 144 rows that don't have IT span metadata

In [ ]:
# show rows where IT span is missing
bf2[bf2['_merge'] == "left_only"]

In [ ]:
np.unique(bf2[bf2['_merge'] == "left_only"]["beeCol"])

In [ ]:
# make sure hive matches, except for the missing bees -- should be 144
np.sum(bf2["Hive"] != bf2["hive"])

In [ ]:
# show all the possible treatments done in the experiments
bf2.groupby(['lowFrq','highFrq']).size().reset_index().rename(columns={0:'count'})

In [ ]:
# add treatment to dataset
trt = []
for ii in range(len(bf2)):
    low = int(float(bf2.lowFrq[ii]))
    high = int(float(bf2.highFrq[ii]))
    if((low == 220.0)  & (high == 450.0)):
        trt.append('full')
    elif(low == 500):
        trt.append('unrewarded')
    elif(low > 330):
        trt.append('high')
    elif((low < 250) & (high <= 350)):
        trt.append('low')
    else:
        trt.append('unknown')
print(len(trt))
print(len(trt) == len(bf2))

In [ ]:
Counter(trt)

In [ ]:

bf2['trt'] = trt

In [ ]:
print(bf2.shape)
bf2.head()

In [ ]:
# save to .csv file
bf2.to_csv('/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/CombinedBeeTrials_noWingbeats.csv', header = True, index = False)

In [ ]:
# impute missing IT spans for 144 rows -- using data from 
# trials when bees were rewarded for the full range -- 220 - 450
smdf = bf2.loc[bf2['trt'] == 'full', :]
len(smdf)

In [ ]:
smdf = smdf.reset_index(drop=True)
smdf.head()

In [ ]:
# show histogram of buzzes
vls = np.array(smdf.freq).reshape(-1,1)
plt.hist(vls, bins = np.arange(215, 455, 10))
plt.show()

In [ ]:
smdf['freq'] = [int(smdf.loc[ii, 'freq']) for ii in range(len(smdf))]

In [ ]:
indBees = pd.DataFrame(smdf.groupby(['beeCol', 'hive'], as_index=False)['freq'].mean())
print(len(indBees))
indBees.head()

In [ ]:
# get bee frequencies (excluding the missing bees)
indFreqs = pd.DataFrame(smdf.groupby(['beeCol', 'hive', 'IT'], as_index=False)['freq'].mean())
print(len(indFreqs))
indFreqs.head()

In [ ]:
# merge datasets
beeFreq = indBees.merge(indFreqs, how = 'outer', indicator = True)
print(len(beeFreq))
beeFreq.head()

In [ ]:
# use linear regression to estimate relationship between avg freq and IT span

# Use only one feature
freq_X_train = np.array(indFreqs.loc[:,"freq"]).reshape(-1,1)
freq_X_test = np.array(beeFreq.loc[beeFreq["_merge"] == 'left_only', "freq"]).reshape(-1,1)

# response variable
freq_y_train = np.array(indFreqs.loc[:, "IT"]).reshape(-1,1)


# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(freq_X_train, freq_y_train)

# Make predictions using the testing set
freq_y_pred = np.round(regr.predict(freq_X_test), 2)
freq_y_pred, 2

In [ ]:
train = plt.scatter(freq_X_train, freq_y_train, label = "raw data")
lne = np.arange(280, 400).reshape(-1,1)
pred, = plt.plot(lne, regr.predict(lne), color = 'green', label='predicted line')
imputedVals = plt.scatter(freq_X_test, freq_y_pred, color = 'r', label = "imputed values")

plt.legend(handles=[pred, train, imputedVals])
plt.show()

###  Put imputed values back into dataset


In [ ]:
naSet = beeFreq.loc[beeFreq["_merge"] == 'left_only', :].copy()
naSet["IT"]= freq_y_pred
naSet = naSet.rename(index=str, columns={"IT": "IT_imputed"})
naSet.head()
naSet = naSet.drop(["_merge", 'freq'], axis = 1)
naSet

In [ ]:
bb2 = beeFreq.copy()

bb2.loc[np.isnan(bb2["IT"]), "IT"] = np.array(naSet["IT_imputed"])
bb2 = bb2.rename(index = str, columns = {"IT": "IT_imputed", "freq": "meanFreq"})
bb2 = bb2.drop("_merge", axis = 1)
bb2.head()

In [ ]:
print(bf2.shape)
bf2.head()

In [ ]:
beeFreq2 = pd.merge(left = bb2, right = bf2.drop("_merge", axis = 1), on = ["beeCol", "hive"], how = 'outer', indicator = "BothDFS")
# fill na's for IT_imputed
beeFreq2['IT_imputed'] = beeFreq2['IT_imputed'].fillna(beeFreq2['IT'])

print(beeFreq2.shape)
beeFreq2.head()

In [ ]:
Counter(beeFreq2.BothDFS) 
# the 240 were bees that were never rewarded 
# for the full range (220 - 450 hz)

In [ ]:
bcs = np.unique(beeFreq2.beeCol)
noFull = []
for ii in bcs:
    levels = np.unique(beeFreq2.loc[beeFreq2.beeCol == ii, "trt"])
    if "full" not in levels:
        noFull.append([ii, len(beeFreq2.loc[beeFreq2.beeCol == ii, "trt"]), levels])

In [ ]:
noFull # these treatments add up to 240

In [ ]:
beeFreq2.drop(["BothDFS", "BeeColorNum", "Hive"], axis = 1).to_csv('/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/CombinedBeeTrials_noWingbeats.csv', header = True, index = False)

In [ ]:
beeFreq2.shape

In [ ]:
np.sum(np.isnan(beeFreq2.IT_imputed))

### Make a clean dataset for analysis

In [ ]:
beeFreq3 = beeFreq2.drop(["BothDFS", "BeeColorNum", "Hive"], axis = 1).copy()

In [ ]:
beeFreq2.groupby(['lowFrq','highFrq']).size().reset_index().rename(columns={0:'count'})

In [ ]:
keep1 = ((pd.to_numeric(beeFreq3["lowFrq"]) == 220) & (pd.to_numeric(beeFreq3["highFrq"]) == 330))
keep2 = ((pd.to_numeric(beeFreq3["lowFrq"]) == 220) & (pd.to_numeric(beeFreq3["highFrq"]) == 450))
keep3 = ((pd.to_numeric(beeFreq3["lowFrq"]) == 338) & (pd.to_numeric(beeFreq3["highFrq"]) == 388))

beeFreq4 = beeFreq3.loc[keep1 | keep2 |keep3, :]

In [ ]:
beeFreq4.head()

In [ ]:
print(beeFreq4.shape)
beeFreq4.groupby(['lowFrq','highFrq']).size().reset_index().rename(columns={0:'count'})

In [ ]:
# remove whitepink, trial 2, full treatment (11 rows that were messed up)
pinkError = ((beeFreq4.beeCol == "whitepink") & (beeFreq4.trialNum == 2) & (beeFreq4.trt == "full"))
beeFreq5 = beeFreq4.loc[~pinkError, :]
beeFreq5.shape

In [ ]:
# remove orange trial 1, high treatment (another error)
orangeError = ((beeFreq5.beeCol == "orange") & (beeFreq5.trialNum == 1) & (beeFreq5.trt == "high"))
beeFreq6 = beeFreq5.loc[~orangeError, :]
beeFreq6.shape

In [ ]:
# make new column that shows amplitude of acceleration in m/s/s
#this image shows the calibration for the accelerometer
Image("/Users/cswitzer/Dropbox/SonicationBehavior/accelerometer_calib.jpeg", width = 300)

In [ ]:
# conversion for acceleration (based on image above)
beeFreq7 = beeFreq6.copy()
beeFreq7["amp_acc"] = (beeFreq7.amp * 1000.0) / 10.17

In [ ]:
beeFreq7.to_csv('/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/01_CombinedTrials_cleaned.csv', header = True, index = False)

In [ ]:
# print system info
import IPython
print(IPython.sys_info())

In [ ]:
# show installed packages and versions
!pip freeze 

In [ ]:
# convert to html, so ppl don't have to run python to see code
os.chdir(baseDir)
!jupyter nbconvert --to html 001_CombineTrialsIntoLongCSV